In [16]:
import os
import glob
import pandas as pd

In [8]:
# Read all csv files in the data folder and subfolders
data_files = glob.glob('../data/**/IVg*.csv', recursive=True)
data = [pd.read_csv(f, comment='#') for f in data_files]

In [65]:
def to_SI(val: str) -> str|float:
    """Transforms a string containing data in SI units, and it transforms
    it to float if possible."""
    if val.endswith((' s', ' V', ' A')):
        val = val[:-2]
    try:
        return float(val)
    except ValueError:
        return val

def get_params(file: str):
    """Get the parameters from the file header. Return a dictionary with the
    parameters.
    """
    dic = dict()
    with open(file) as f:
        for line in f:
            if not line.startswith('#'):
                break

            key, value = line.strip('#').strip().split(':')
            if not key in ['Parameters', 'Data']:
                dic[key] = to_SI(value.strip())
            if key == 'Procedure':
                # extract 'procedure' from '<__name__.procedure>'
                procedure = value.strip().split('.')[-1][:-1]
                dic['Procedure'] = procedure
                dic['Date'] = file.split(procedure)[-1][:10]
            dic['Path'] = os.path.relpath(file)
    return dic

In [67]:
param_df = pd.DataFrame([get_params(f) for f in data_files])
param_df.set_index(['Procedure', 'Path'], inplace=True)
param_df.sort_index(inplace=True)

param_df[param_df['Date'] == '2023-05-04']

Date  Irange  N_avg   
Procedure Path                                                                
IVg       ..\data\2023-05-04\IVg2023-05-04_1.csv  2023-05-04   0.001    2.0  \
          ..\data\2023-05-04\IVg2023-05-04_2.csv  2023-05-04   0.001    2.0   
          ..\data\2023-05-04\IVg2023-05-04_3.csv  2023-05-04   0.001    2.0   

                                                    Chip Sample  Step time   
Procedure Path                                                               
IVg       ..\data\2023-05-04\IVg2023-05-04_1.csv  testtt      J       0.01  \
          ..\data\2023-05-04\IVg2023-05-04_2.csv  testtt      J       0.01   
          ..\data\2023-05-04\IVg2023-05-04_3.csv  testtt      J       1.00   

                                                    VDS  VG end  VG start   
Procedure Path                                                              
IVg       ..\data\2023-05-04\IVg2023-05-04_1.csv  0.075    35.0     -35.0  \
          ..\data\2023-05-04\IVg2023-05-04_2.csv  0.075     5.0      -5.0   
          ..\data\2023-05-04\IVg2023-05-04_3.csv  0.075     2.0      -2.0   

                                                  VG step Comment  
Procedure Path                                                     
IVg       ..\data\2023-05-04\IVg2023-05-04_1.csv      0.2       -  
          ..\data\2023-05-04\IVg2023-05-04_2.csv      0.2       -  
          ..\data\2023-05-04\IVg2023-05-04_3.csv      0.2       -